In [40]:
import pandas as pd

train = pd.read_csv('coursera_sessions_train.txt', sep = ';', header = None)
train.columns = ['watch', 'buy']
train.head()

,watch,buy
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


In [41]:
df1 = pd.DataFrame(columns = ['id', 'cnt'])
df1.head()

,id,cnt


In [42]:
d_tr_wcnt = dict() # словарь кол-ва просмотров
d_tr_wrow = dict() # словарь первой строки
for ind, sess in enumerate(train.watch):
    for item in sess.split(','):
        d_tr_wcnt[item] = d_tr_wcnt.get(item, 0) + 1
        d_tr_wrow[item] = d_tr_wrow.get(item, ind)

d_tr_w = []
for key in d_tr_wcnt.keys():
    d_tr_w.append([key, d_tr_wcnt[key], d_tr_wrow[key]])

dfw = pd.DataFrame(data = d_tr_w, columns = ['id', 'cnt', 'idx'], dtype = int)
dfw.head()

,id,cnt,idx
0,50088,1,25480
1,58146,2,15269
2,44884,1,9755
3,13356,1,20438
4,73399,2,24589


In [43]:
d_tr_bcnt = dict() # словарь кол-ва покупок
d_tr_brow = dict() # словарь первой строки покупки
for ind, sess in enumerate(train.buy.dropna()):
    for item in sess.split(','):
        d_tr_bcnt[item] = d_tr_bcnt.get(item, 0) + 1
        d_tr_brow[item] = d_tr_brow.get(item, ind)

d_tr_b = []
for key in d_tr_bcnt.keys():
    d_tr_b.append([key, d_tr_bcnt[key], d_tr_brow[key]])

dfb = pd.DataFrame(data = d_tr_b, columns = ['id', 'cnt', 'idx'], dtype = int)
dfb.head()

,id,cnt,idx
0,50088,1,1848
1,19719,1,3276
2,16303,2,2199
3,35233,1,1299
4,35546,1,1427


In [44]:
dfw.sort_values(['cnt', 'idx'], ascending = [False, True], inplace = True)


In [45]:
dfb.sort_values(['cnt', 'idx'], ascending = [False, True], inplace = True)

In [54]:
import numpy as np
wrec5 = []
wrec1 = []
wprec5 = []
wprec1 = []

brec5 = []
brec1 = []
bprec5 = []
bprec1 = []

tdna = train.dropna()
for sess in tdna.values:
    watch = [int(x) for x in sess[0].split(',')]
    buy = [int(x) for x in sess[1].split(',')]
    
    top5w = dfw[dfw.id.isin(watch)].id.values[:5]
    top1w = set(top5w[0:1])
    top5w = set(top5w)
    
    buy_set = set(buy)
    lb = len(buy_set)
    li5 = float(len(top5w.intersection(buy_set)))
    wprec5.append(li5 / 5.)
    wrec5.append(li5 / lb)
    
    li1 = float(len(top1w.intersection(buy_set)))
    wprec1.append(li1)
    wrec1.append(li1 / lb)
    
    top5b = dfb[dfb.id.isin(watch)].id.values[:5]
    top1b = set(top5b[0:1])
    top5b = set(top5b)
    
    li5 = float(len(top5b.intersection(buy_set)))
    bprec5.append(li5 / 5.)
    brec5.append(li5 / lb)
    
    li1 = float(len(top1b.intersection(buy_set)))
    bprec1.append(li1)
    brec1.append(li1 / lb)
    

answ = ['%.2f'%x for x in np.mean([wrec1, wprec1, wrec5, wprec5], axis = 1)]
ansb = ['%.2f'%x for x in np.mean([brec1, bprec1, brec5, bprec5], axis = 1)]

print answ, ansb
with open('l1.txt', 'w') as f:
    f.write(" ".join(answ))
with open('l2.txt', 'w') as f:
    f.write(" ".join(ansb))

['0.44', '0.50', '0.82', '0.21'] ['0.68', '0.79', '0.93', '0.25']


In [52]:

test = pd.read_csv('coursera_sessions_test.txt', sep = ';', header = None)
test.columns = ['watch', 'buy']
test.head()

,watch,buy
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [61]:
wrec5 = []
wrec1 = []
wprec5 = []
wprec1 = []

brec5 = []
brec1 = []
bprec5 = []
bprec1 = []

tdna = test.dropna()


for sess in tdna.values:
    watch = [int(x) for x in sess[0].split(',')]
    buy = [int(x) for x in sess[1].split(',')]
    
    top5w = dfw[dfw.id.isin(watch)].id.values[:5]
    if top5w.shape[0] < 5:
        neww = set(watch).difference(set(top5w))
        top5w = np.append(top5w, list(neww))[:5]
    top1w = set(top5w[0:1])
    top5w = set(top5w)
    
    buy_set = set(buy)
    lb = len(buy_set)
    li5 = float(len(top5w.intersection(buy_set)))
    wprec5.append(li5 / 5.)
    wrec5.append(li5 / lb)
    
    li1 = float(len(top1w.intersection(buy_set)))
    wprec1.append(li1)
    wrec1.append(li1 / lb)
    
    top5b = dfb[dfb.id.isin(watch)].id.values[:5]
    if top5b.shape[0] < 5:
        newb = set(watch).difference(set(top5b))
        top5b = np.append(top5b, list(newb))[:5]
    top1b = set(top5b[0:1])
    top5b = set(top5b)
    
    li5 = float(len(top5b.intersection(buy_set)))
    bprec5.append(li5 / 5.)
    brec5.append(li5 / lb)
    
    li1 = float(len(top1b.intersection(buy_set)))
    bprec1.append(li1)
    brec1.append(li1 / lb)
    

answ = ['%.2f'%x for x in np.mean([wrec1, wprec1, wrec5, wprec5], axis = 1)]
ansb = ['%.2f'%x for x in np.mean([brec1, bprec1, brec5, bprec5], axis = 1)]
print answ, ansb
with open('l3.txt', 'w') as f:
    f.write(" ".join(answ))
with open('l4.txt', 'w') as f:
    f.write(" ".join(ansb))

['0.41', '0.48', '0.80', '0.20'] ['0.42', '0.48', '0.79', '0.20']
